# Configuración

In [3]:
# API_KEY = "sk-proj-***-jaB6O2LeSsrQ1YQA"

In [7]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


llm_model = OpenAI(
    model="gpt-4o-mini",
    api_key=API_KEY
)

embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    api_key=API_KEY
)


Settings.llm = llm_model
Settings.embed_model = embed_model

# RAG con pocas líneas

In [8]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("seguros").load_data()

In [10]:
len(documents)

17

In [13]:
# documents[1]

In [14]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [15]:
query_engine = index.as_query_engine()

In [16]:
response = query_engine.query("¿Cuál es la protección jurídica del seguro de hogar?")

In [18]:
response.response

'La protección jurídica del seguro de hogar incluye asesoramiento telefónico ilimitado y defensa en reclamaciones derivadas del contrato de alquiler. También se ofrece asistencia para la obtención de documentación y servicios jurídicos y extrajudiciales, excepto en procedimientos de desahucio si se ha dejado de pagar el alquiler.'

In [21]:
len(response.source_nodes)

2

In [22]:
response.metadata

{'0fc05f59-2cf7-4b4e-9494-fd7917e80663': {'page_label': '6',
  'file_name': 'AF-SL-TablasGarantiasHogar-Completo-Modalidad14-v11-CAS.pdf',
  'file_path': 'C:\\Users\\efclprd\\Desktop\\nlp\\seguros\\AF-SL-TablasGarantiasHogar-Completo-Modalidad14-v11-CAS.pdf',
  'file_type': 'application/pdf',
  'file_size': 96118,
  'creation_date': '2025-07-04',
  'last_modified_date': '2025-07-04'},
 '03cd006a-22f7-47ff-a748-2ddf6eaec9f3': {'page_label': '7',
  'file_name': 'AF-SL-TablasGarantiasHogar-Premium-Modalidad15-v13-CAS.pdf',
  'file_path': 'C:\\Users\\efclprd\\Desktop\\nlp\\seguros\\AF-SL-TablasGarantiasHogar-Premium-Modalidad15-v13-CAS.pdf',
  'file_type': 'application/pdf',
  'file_size': 101016,
  'creation_date': '2025-07-04',
  'last_modified_date': '2025-07-04'}}

# Evaluation (LLM-as-a-judge)

In [29]:
from openai import OpenAI
# API_KEY = "sk-proj-f***-jaB6O2LeSsrQ1YQA"
client = OpenAI(api_key=API_KEY)

In [30]:
gt_q = [
     "tengo alguna garantía que cubra los humos?",
"qué seguros de hogar cubren las placas solares?",
"la vitrocerámica está incluida en alguna modalidad del seguro?"
]

In [31]:
gt_a = [
      "Sí, las modalidades de los seguros Hogar Completo, Hogar Eficaz y Hogar Premium incluyen la cobertura de daños ocasionados por humo, ya sea por escapes repentinos en cocinas, sistemas de calefacción u otros aparatos eléctricos.",
"La modalidad Hogar Completo cubre el robo de placas solares en cubiertas de viviendas unifamiliares hasta 3.000 € por incidente. En la modalidad Hogar Premium, la cobertura para el robo de placas solares se amplía hasta 5.000 € por incidente.",
"Sí, las modalidades Hogar Completo y Hogar Premium cubren la rotura accidental de elementos vitrocerámicos de cocina, incluyendo la reposición de la placa o cristal."

]

In [32]:
llm_a = []
for qq in gt_q:
    rr = query_engine.query(qq)
    llm_a.append(rr.response)
    print(rr.response)
    print("*"*50)

No se menciona específicamente una garantía que cubra los daños por humos en la información proporcionada. Sin embargo, se incluyen coberturas para daños por incendio, explosión o daños por agua, que podrían ser relevantes dependiendo de la situación. Es recomendable revisar los términos y condiciones específicos del seguro para obtener detalles sobre las coberturas disponibles.
**************************************************
Los seguros de hogar que cubren las placas solares indemnizan en caso de robo de las mismas, con una cobertura de hasta 5.000 € por incidente y anualidad de seguro.
**************************************************
La vitrocerámica no se menciona específicamente en las garantías opcionales descritas. Sin embargo, si se produce un daño en la vitrocerámica debido a un incidente cubierto por el seguro, podría estar incluida en la cobertura de daños producidos por la electricidad o en otras garantías relacionadas con daños a electrodomésticos. Es recomendable revi

In [37]:
def evalute_question(ga, llma):
    
    grading_prompt = f"""
    Evalúa la calidad de la respuesta generada por el modelo LLM en comparación con \
    la respuesta de referencia (ground truth). \
    A continuación se presentan los criterios para la evaluación: 
    - **Score 1:** La respuesta del LLM es irrelevante o incorrecta en comparación con \
    la verdad de referencia (ground truth). 
    - **Score 2:** La respuesta del LLM es algo relacionada pero tiene errores significativos \
    o está incompleta en comparación con la verdad de referencia. 
    - **Score 3:** La respuesta del LLM es aceptable, pero carece de precisión o detalles \
    importantes en comparación con la verdad de referencia. 
    - **Score 4:** La respuesta del LLM es buena y responde adecuadamente, aunque hay detalles \
    que podrían mejorarse para alinearse más con la verdad de referencia. 
    - **Score 5:** La respuesta del LLM es excelente, clara, completa y tan precisa como la \
    verdad de referencia o incluso aporta detalles adicionales útiles. 
    
    Proporciona un puntaje del 1 al 5. Asegurate de que solo respondes un valor entre el 1 y el 5. 
    
    **Ground truth:** {ga}
    
    **Respuesta del LLM:** {llma} 
    
    **Evaluación:** 
    """
    
    completion = client.chat.completions.create(
        model = 'gpt-4o',
        messages=[
            {"role": "user", "content": grading_prompt}
        ]
    )
    
    return completion.choices[0].message.content

In [42]:
import numpy as np

scores = []
for ga, llma, in zip(gt_a, llm_a):
    rr = evalute_question(ga, llma)
    print(rr)
    scores.append(int(rr))
np.mean(scores)

1
2
2


1.6666666666666667

# Mejoras

In [58]:
import nest_asyncio
nest_asyncio.apply()

In [73]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import QuestionsAnsweredExtractor, SummaryExtractor

text_splitter = SentenceSplitter(chunk_size=300, chunk_overlap=50)
summ = SummaryExtractor(summaries=['self'])
qa = QuestionsAnsweredExtractor(questions=2)

In [74]:
summ.prompt_template
qa.prompt_template = "Aquí está el contexto:\n{context_str}\n\nDada la informacion contextual, genera {num_questions} y especifica el tipo de seguro Hogar Completo, Hogar Eficaz, Hogar Premium"
qa.prompt_template

'Aquí está el contexto:\n{context_str}\n\nDada la informacion contextual, genera {num_questions} y especifica el tipo de seguro Hogar Completo, Hogar Eficaz, Hogar Premium'

In [75]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=[
    text_splitter,
#     summ,
    qa
])

In [76]:
nodes = pipeline.run(documents=documents)

100%|██████████████████████████████████████████████████████████████| 105/105 [02:30<00:00,  1.43s/it]


In [77]:
len(nodes)

105

In [78]:
# nodes[4]

In [79]:
nodes[4].metadata

{'page_label': '2',
 'file_name': 'AF-SL-TablasGarantiasHogar-Completo-Modalidad14-v11-CAS.pdf',
 'file_path': 'C:\\Users\\efclprd\\Desktop\\nlp\\seguros\\AF-SL-TablasGarantiasHogar-Completo-Modalidad14-v11-CAS.pdf',
 'file_type': 'application/pdf',
 'file_size': 96118,
 'creation_date': '2025-07-04',
 'last_modified_date': '2025-07-04',
 'questions_this_excerpt_can_answer': 'Con base en la información proporcionada, aquí tienes dos ejemplos de coberturas para diferentes tipos de seguros de hogar:\n\n1. **Seguro Hogar Completo**:\n   - **Cobertura por Caída de Árboles, Postes y Antenas**: Este seguro cubre los daños que sufra la vivienda y sus contenidos debido a la caída de árboles, postes o antenas. Por ejemplo, si la antena de tu tejado se cae y daña la bicicleta en el jardín o el alfeizar de la ventana, la reparación de ambos estará asegurada. Sin embargo, la antena en sí no estará cubierta.\n   - **Cobertura por Fenómenos Meteorológicos**: Los daños causados por viento, pedrisco o

In [83]:
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(similarity_top_k=4)

In [84]:
# genero nuevas respuestas

llm_a = []
for qq in gt_q:
    rr = query_engine.query(qq)
    llm_a.append(rr.response)
#     print(rr.response)
#     print("*"*50)

In [85]:
# evaluo

scores = []
for ga, llma, in zip(gt_a, llm_a):
    rr = evalute_question(ga, llma)
    print(rr)
    scores.append(int(rr))
np.mean(scores)

4
5
5


4.666666666666667

In [87]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(zip(gt_q, gt_a, llm_a, scores), columns=['gt_question', 'gt_answer', 'rag_answer', 'score'])
df

,gt_question,gt_answer,rag_answer,score
0,tengo alguna garantía que cubra los humos?,"Sí, las modalidades de los seguros Hogar Completo, Hogar Eficaz y Hogar Premium incluyen la cobertura de daños ocasionados por humo, ya sea por escapes repentinos en cocinas, sistemas de calefacción u otros aparatos eléctricos.","Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías se activan en caso de escapes repentinos de humo en cocinas, sistemas de calefacción y otros aparatos eléctricos. Por ejemplo, si se produce una fuga de humo que ennegrece una pared, el seguro se encargará de los costos de limpieza y reparación.",4
1,qué seguros de hogar cubren las placas solares?,"La modalidad Hogar Completo cubre el robo de placas solares en cubiertas de viviendas unifamiliares hasta 3.000 € por incidente. En la modalidad Hogar Premium, la cobertura para el robo de placas solares se amplía hasta 5.000 € por incidente.","Los seguros de hogar que cubren las placas solares incluyen el Seguro Hogar Completo y el Seguro Hogar Premium. Ambos ofrecen indemnización en caso de robo de placas solares ubicadas en la cubierta de la vivienda, garantizando hasta 3,000 € por incidente en el caso del Seguro Hogar Completo y hasta 5,000 € por incidente en el Seguro Hogar Premium.",5
2,la vitrocerámica está incluida en alguna modalidad del seguro?,"Sí, las modalidades Hogar Completo y Hogar Premium cubren la rotura accidental de elementos vitrocerámicos de cocina, incluyendo la reposición de la placa o cristal.","Sí, la cobertura de la vitrocerámica está incluida en las modalidades de seguro ""Hogar Completo"" y ""Hogar Premium"". Ambas modalidades cubren la rotura accidental de la placa o cristal de la vitrocerámica, así como los gastos de reposición y colocación de un nuevo cristal en caso de que se rompa.",5


# External VEctor store

In [88]:
index.storage_context.persist("vector_store")

In [90]:
from llama_index.core import StorageContext, load_index_from_storage

sc = StorageContext.from_defaults(persist_dir="vector_store")

In [91]:
index2 = load_index_from_storage(sc)
query_engine2 = index2.as_query_engine()

In [97]:
rr = query_engine2.query(gt_q[0])
rr.response

'Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías protegen tanto la vivienda como los bienes contenidos en ella, incluyendo daños causados por humo que provenga de incendios, explosiones o incluso incidentes menores, como una llamarada en la cocina. Esto significa que si el humo daña elementos como la campana extractora o el techo, los costos de reparación y restauración estarían cubiertos.'